<a href="https://colab.research.google.com/github/JancerGomes/HR-diversity-overview/blob/master/Human_Resources_Current.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Resources Diversity Analysis

In this notebook - my debut project - we'll be analyzing a Human Resources synthetic dataset made available on Kaggle, from a make-believe company called Dental Magic.
I have selected some of the questions provided in the dataset page (https://www.kaggle.com/rhuebner/human-resources-data-set?rvi=1), here they are:

1. What are our best recruiting sources if we want to ensure a diverse organization?
2. Are there areas of the company where pay is not equitable?
3. What is the overall diversity profile of the organization?

We will be analyzing each matter with regards to race, sex, immigration status and age.

## Importing essential libraries
These are standard libraries for EDA. Here we go:

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Exploring the data
### First Look

In [3]:
hr_data = pd.read_csv('humanresources.csv')
hr_data.info()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Lots of empty rows. Some cleaning now:
hr_data.dropna(how='all', inplace=True)

### Racial Diversity

In [ ]:
# A first look at the racial diversity situation - both in absolute and percentual figures:
display(hr_data.RaceDesc.value_counts(),
        hr_data.RaceDesc.value_counts(normalize=True) * 100)

According to 2018 data by the U.S Bureau of Labor Statistics, the country's labor force is made of 78% Whites, 13% Blacks and 6% Asians (https://www.bls.gov/opub/reports/race-and-ethnicity/2018/home.htm). Taking that as a baseline, we can say that the company is diverse from a racial standpoint.

**Let's see which recruitment sources hire the most non-white employees:**

In [ ]:
hr_data['Non-white'] = (hr_data['RaceDesc'] != 'White')
fig = plt.figure(figsize=(6, 6)), sns.set_style('whitegrid')
ax = sns.barplot(data=hr_data, y='RecruitmentSource', x='Non-white', ci=None, palette='Set2')
ax.set_xlabel('Non-white proportion'), ax.set_ylabel('Recruitment Source');

Unsurprisingly, **Diversity Job Fair** plays a crucial role in promoting racial diversity. **Indeed and On-campus Recruiting** can also be lauded for bringing non-white employees more often than not. 

On the other hand, **Pay Per Click, On-line Web application, Careerbuilder and Company Intranet** have no contribution to racial diversity at all.


Of course, **respect to diversity includes egalitarian compensation, regardless of skin color.** Let's take a look at that now.

In [ ]:
plt.figure(figsize=(6, 6))
ax = sns.boxplot(data=hr_data, x='RaceDesc', y='PayRate', palette='Set2')
ax.set_xticklabels(hr_data.RaceDesc.unique(), rotation=90)
sns.pointplot(data=hr_data, x='RaceDesc', y='PayRate', join=False, ci=None, ax=ax);
ax.set_xlabel('Races'), ax.set_ylabel('Hourly pay rate');

There are a few differences in median and mean payment across races. For instance, Hispanics are placed a bit higher. This does _not_, however, seem particularly problematic. Let's look at each department.

In [ ]:
g = sns.catplot(data=hr_data, x='RaceDesc', col_wrap=2,
                y='PayRate', col='Department', kind='box', palette='Set2')
g.set_xticklabels(hr_data.RaceDesc.unique(), rotation=90)
g.set_xlabels('Race'), g.set_ylabels('Hourly pay rate')
g.fig.suptitle(
    'Pay rate ditribution by race group in each department', fontsize=16)
g.set_titles('{col_name}')
plt.subplots_adjust(top=0.95)

An in-depth look at each department shows that payment is generally egalitarian. **The exception is *IT/IS*, wherein those of *two or more races* have a diminished pay rate.** Why is that? Are these workers in low-paid positions?

In [ ]:
ax = sns.scatterplot('PayRate', 'Position', 'RaceDesc', data=hr_data.loc[hr_data.Department=='IT/IS'],
                     palette='Set2', style='RaceDesc',size='RaceDesc', sizes=[40, 40, 40, 120, 40]);
ax.legend(bbox_to_anchor=(1, 1)); ax.set_xlabel('Hourly pay rate');

The plot reveals that employees of this race group are actually being paid lower than colleagues in the same position. This could be the result of discriminatory practices. Let's investigate further to see if there are any reasonable causes for the wage gap:

In [ ]:
# Filtering only rows that contain "two or more races" workers
position_rows = hr_data.Position.isin(['IT Support', 'IT Manager - DB'])
perf_indicators = ['RaceDesc','Position', 'PerformanceScore', 'SpecialProjectsCount', 'DaysLateLast30','EngagementSurvey']
it_is_lookup = hr_data.loc[position_rows, perf_indicators].sort_values(['Position','RaceDesc']).set_index('RaceDesc')
it_is_lookup

The perfomance of *two or more races* employees is rather levelled with that of colleagues in the same position, yet their pay rates are the lower than any other. **As far as the dataset goes, this points to the existence of discrimination.**

---
**DISCLAIMER: here and throughout the end of this analysis, there is an observation that distinguishes itself from the others: at Executive Office, we find the President & CEO of the company, who is naturally the most well-paid. Take a look at her:**

In [ ]:
hr_data.loc[hr_data['Department'] == 'Executive Office'] 

### Gender Equality

In [ ]:
# For clarity:
hr_data.replace({'Sex': {'F': 'Female', 'M ': 'Male'}}, inplace=True)
# Now, to an overview in gender distribution:
display(hr_data.Sex.value_counts(),'\n', (hr_data.Sex.value_counts(normalize=True) * 100))

The numbers are balanced, and **most employees are female.** 

Taking a look at each department now:

In [ ]:
# By department
g = sns.catplot(data=hr_data, x='Sex', col='Department',
                col_wrap=2, palette='Spectral', kind='count')
g.set_xlabels('Gender'), g.set_ylabels('# of employees')
g.fig.suptitle(
    'Employee count by gender in each department', fontsize=16)
g.set_titles('{col_name}')
plt.subplots_adjust(top=0.95)

We can see that the numeric difference stems from the *Production department*, where women outnumber men by 50%.

**But are female employees getting equal pay?**

In [ ]:
GenderPay = hr_data.groupby('Sex')[['PayRate']]
display(GenderPay.agg(['mean', 'median']))

**That's a *no*.** Women's income is lower in general.

How does that distribute across departments?

In [ ]:
# Removing whitespace in 'Production':
hr_data['Department'] = hr_data['Department'].str.strip()
plt.figure(figsize=(8, 6))
ax = sns.pointplot(data=hr_data, x='Department', y='PayRate',
                   hue='Sex', palette='Spectral' , join=False)
ax.set_xticklabels(hr_data.Department.unique(), rotation=45,
                   horizontalalignment='right');
ax.set_title("Average pay rate by department"), ax.set_ylabel(
    'Hourly pay rate');

At the plot, we identify some divergences in average pay by gender. Women's average pay is quite higher at Software Engineering department. However, it is important to note that only a handful of employees work there, in such a way that it doesn't affect the overall statistics so much.

On the other hand, *Production* plays a big part in broadening the wage gap because:
1. the department has the lowest pay rate; 
2. this decreases female mean compensation by a lot, **since it contains the largest number of workers, most of them being women**;
3. considering the employee amount, even though the inequality within Production itself isn't great, it ends up making a significant impact.

**We can see a substantial wage gap at Admin Offices**; there, women's average income is much lower than men's.

It demands some examining:

In [ ]:
ax = sns.stripplot(x='Position', y='PayRate', hue='Sex',
                   data=hr_data[hr_data.Department == 'Admin Offices'],
                  palette='Spectral')
ax.set_ylabel('Pay rate'), ax.set_title('Pay rate at Production department')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right');

An in-depth analysis reveals that the wage disparity at Admin Offices, the largest throughout departments, comes from the presence of an outlier: a male worker that acts as Shared Services Manager. 

Among those whose title is Accountant I, the female employee's pay rate is only marginally lower than that of a male colleague. We can also see that only women work as Sr. Accountants, **which shows that females aren't kept away from higher positions in this department**.


### Citizenship status
For starters, we look at the employee distribution by citizenship status:

In [ ]:
# A bit of organization
hr_data.replace({'CitizenDesc': {'US Citizen': 'US citizen',
                                 'Eligible NonCitizen': 'Eligible non-citizen', 'Non-Citizen': 'Non-citizen'}}, inplace=True)
hr_data['CitizenDesc'].value_counts()

In [ ]:
# Considering that the large majority of employees is composed of US citizens,
# let's see how the few non-citizens are hired:
ax = sns.countplot(data=hr_data.loc[hr_data['CitizenDesc'] != 'US citizen'],
                   x='CitizenDesc', hue='RecruitmentSource', palette='Paired')
ax.legend(bbox_to_anchor=(1, 1)),
ax.figure.set_size_inches((5, 6))
ax.set_xlabel('Citizenship status')
ax.set_yticks([0, 1, 2]);

Once again, *Diversity Job Fair* scores the highest. *Social networks, employee referral and search engines* also play their part in hiring non-citizens in general.
Anyhow, **Dental Magic is not really diverse regarding this aspect.**

How is the payroll configured when it comes to citizenship status?

In [ ]:
ax = sns.swarmplot(data=hr_data, x='Department', y='PayRate',
                   hue='CitizenDesc', palette='Set2')
ax.figure.set_size_inches((7, 5))
ax.legend(title='Citizenship status'), ax.set_ylabel('Pay rate')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45,
                   horizontalalignment='right');

The hourly pay rate for each non-citizen is always paired or very close to that of a US citizen — presumably a co-worker in a similar position. This leads to the conclusion that there is no wage gap between the groups.

### Age statistics
Now, you might have noted that there's no age info in the dataset. Well, at least *not yet*.

We can obtain it through the date of birth (DOB) column

In [ ]:
# We can make our own age column.
# But first, let's convert 'DOB' to datetime format, with a bit of chaining to keep just the date
from dateutil.relativedelta import relativedelta
hr_data['DOB'] = pd.to_datetime(hr_data['DOB']).dt.date.astype('datetime64')
# No employees were born after year 2000, so DOBs like 2068 should have 100 years removed:
hr_data.loc[hr_data.DOB.dt.year > 2000, 'DOB'] -= pd.DateOffset(years=100)
# Now, to getting the age:
hr_data['Age'] = pd.Series(dtype='int')
for ind, date in hr_data.DOB.iteritems():
    hr_data.loc[ind, 'Age'] = relativedelta(
        pd.to_datetime('today'), date).years
hr_data.Age = hr_data.Age.astype('int64')
#Some summary statistics
hr_data['Age'].apply(['min', 'median', 'max'])

Now we have the employees' ages stored in a column. They range from 27 to 69; the median is 39.

In [ ]:
ax = sns.distplot(hr_data['Age'], kde=False)

The staff's ages are mainly at early to mid-30s, also counting high at early 40s.
The number of employees is substantially lower from ages 50 and over, amounting only to no more than a few dozens.

Considering how low the numbers go when it comes to older workers, it's important that we inquire how age diversity is promoted through recruiting.

In [ ]:
ax = sns.countplot(data=hr_data.loc[hr_data['Age'] > 50],
                   y='RecruitmentSource')
ax.set_ylabel('Recruitment Source'), ax.set_xlabel('Employee count')
ax.set_title('# of employees (ages 50+)');

Again, **Diversity Job Fair** scores the highest, tied with **employee referral**. Search **engines, billboards and Monster.com** also brought some of the older employees.

**Are salaries subject to *ageism* at Dental Magic?** It is necessary to take a look at how mature workers are being payed.

In [ ]:
g = sns.relplot('Age', 'PayRate', col='Department', col_wrap=2, data=hr_data, color='purple')
g.set_xlabels('Age'), g.set_ylabels('Hourly pay rate')
g.fig.suptitle(
    'Age x Pay rate across departments', fontsize=16)
g.set_titles('{col_name}')
plt.subplots_adjust(top=0.95)

We saw before that the employee count goes down for ages 50 and over. In *Production*, we can see that **salaries also go down for employees who are about 55 or older**: they have a pay rate below 35, whereas some younger workers get as high as $60/hour. This calls for a more detailed analysis of this specific division.

In [ ]:
agegroups= pd.cut(hr_data.Age, [25,40,55,70])
ax = sns.boxplot('PayRate', 'Position', agegroups, 
                   data=hr_data.loc[hr_data.Department=='Production'], palette='Dark2') ;
ax.set_xlabel('Hourly pay rate');

The graph clarifies that it is generally not the case that older workers are getting a lower salary than their position's average. **Matures' pay rates are limited because almost all of them work in lesser functions (Production Technician I and II).**

**In *Software Engineering*, there is an outlier: a worker just over 50 years old, whose pay rate is much lower than that of younger colleagues**. We will now take a closer look at the department to shed light on the matter.

In [ ]:
# The department:
soft_engs = hr_data.Department == 'Software Engineering'
# Columns for analysis:
relevant_info = ['Employee_Name', 'Age', 'Position', 'PayRate',
                 'PerformanceScore', 'EmploymentStatus', 'EngagementSurvey']
# Now, we select them and look at some relevant columns:
hr_data.loc[soft_engs, relevant_info].sort_values('Age', ascending=False).set_index('Age')

We now see that the outlier is Alex Sweetwater. 

He is the manager of the department, but his pay is registered as the lowest one. It is highly unlinkely that his pay rate is lower than his subordinates'. Also, the data doesn't point to bad performance. **It may be sensible to assume that this piece of information is actually *incorrect*, possibly due to input error.**

## Sharing the findings
 

### 1. What are our best recruiting sources if we want to ensure a diverse organization?

In order to answer this question, mainly on the variables in which the company has enormous lack of diversity.

### 2. Are there areas of the company where pay is not equitable?

A deep analysis has showed some wage gaps inside departments:

1. **At *IT/IS*, people of *two or more races* are paid significantly less than other workers in the same job position**. None of the data show a cause for that.

2. **Women's income is lower overall.** The problem comes mainly from two sources:
  - in *Production*, the least well-paid department, females outnumber males by some extent;
  - in *Admin Offices*, the wage gap between genders is substantial, though the distribution isn't necessarily discriminatory.

3. Most workers aged 50+ in *Production* work at lower-paid positions.






### 3. What is the overall diversity profile of the organization?

Dental magic can be classified as a diverse company regarding many aspects.

**Race**

While more than half of the workforce is made of white people, we've seen before that the jobs are more well distributed along underrepresented groups than comparatively to official statistics in a national level. 

*Hispanics* get a slightly higher pay rate in average, whereas *American Indian or Alaska Natives* perform lower on that variable. Considering that these groups have only four members each, it is premature to conclude the divergence stems from any discriminatory treatment

**Gender**

The workforce is predominantly female. Additionally, Dental Magic is run by a woman. While these aspects are present, and could be seen as desirable in a world that favors hiring male workers for most positions, the organization still has some problems:


